In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [2]:
xls = pd.ExcelFile('Supplier Lead Times.xlsx')
default_lead = pd.read_excel(xls, 'Default Lead')
stock_lead1 = pd.read_excel(xls, 'Stock Lead')
stock_lead2 = pd.read_excel(xls, 'Sheet4')

In [3]:
default_lead.shape

(61, 3)

In [4]:
stock_lead1.shape

(490499, 10)

In [5]:
stock_lead2.shape

(528518, 10)

In [6]:
# check and drop duplicates
default_lead.drop_duplicates(inplace = True)

In [7]:
stock_lead1.duplicated().value_counts()

False    490499
dtype: int64

In [8]:
stock_lead2.duplicated().value_counts()

False    528518
dtype: int64

In [9]:
default_lead.head()

,Supplier,LoadingPort,TotalLeadTime
0,JADD010,NINGBO,105
1,JBEN020,NINGBO,120
2,JBRT010,NINGBO,120
3,JCDA010,NINGBO,90
4,JRCC010,NaN,30


In [10]:
stock_lead1.head()

,StockCode,Supplier,LoadingPort,DestinationPort,Warehouse,ToSupplierDays,FactoryDeliveryDays,VesselDays,ToWarehouseDays,TotalLeadTime
0,.K6RTC,JTOR020,NINGBO,CAPE TOWN,C2,1,90,33,2,126
1,.K6RTC,JTOR020,NINGBO,DURBAN,B3,1,90,24,5,120
2,.K6RTC,JTOR020,NINGBO,DURBAN,CW,1,90,24,5,120
3,.K6RTC,JTOR020,NINGBO,DURBAN,D1,1,90,24,4,119
4,.K6RTC,JTOR020,NINGBO,DURBAN,J1,1,90,24,5,120


In [11]:
stock_lead2.head()

,StockCode,Supplier,LoadingPort,DestinationPort,Warehouse,ToSupplierDays,FactoryDeliveryDays,VesselDays,ToWarehouseDays,TotalLeadTime
0,HD0700FD02,JHON010,KAOHSIUNG,CAPE TOWN,C2,2,45,29,2,78
1,HD0700FD02,JHON010,KAOHSIUNG,DURBAN,B3,2,45,24,5,76
2,HD0700FD02,JHON010,KAOHSIUNG,DURBAN,CW,2,45,24,5,76
3,HD0700FD02,JHON010,KAOHSIUNG,DURBAN,D1,2,45,24,4,75
4,HD0700FD02,JHON010,KAOHSIUNG,DURBAN,J1,2,45,24,5,76


In [12]:
stock_lead1.Warehouse.unique()

array(['C2', 'B3', 'CW', 'D1', 'J1', 'J2', 'J3', 'L2', 'N1', 'P2', 'E1',
       'EL', 'G1', 'L1'], dtype=object)

In [13]:
stock_lead2.Warehouse.unique()

array(['C2', 'B3', 'CW', 'D1', 'J1', 'J2', 'J3', 'L2', 'N1', 'P2', 'E1',
       'EL', 'G1', 'L1'], dtype=object)

In [14]:
# NO data for J0, A1 warehouses

In [15]:
# sperate data tables for different warehouses

In [16]:
C2_stock_lead1 = stock_lead1[stock_lead1.Warehouse == 'C2']
C2_stock_lead2 = stock_lead2[stock_lead2.Warehouse == 'C2']

In [17]:
C2_stock_lead = pd.concat([C2_stock_lead1,C2_stock_lead2]).reset_index(drop=True)

In [18]:
C2_stock_lead.shape

(85711, 10)

In [19]:
J1_stock_lead1 = stock_lead1[stock_lead1.Warehouse == 'J1']
J1_stock_lead2 = stock_lead2[stock_lead2.Warehouse == 'J1']

In [20]:
J1_stock_lead = pd.concat([J1_stock_lead1,J1_stock_lead2]).reset_index(drop=True)

In [21]:
J1_stock_lead.shape

(102783, 10)

In [22]:
J2_stock_lead1 = stock_lead1[stock_lead1.Warehouse == 'J2']
J2_stock_lead2 = stock_lead2[stock_lead2.Warehouse == 'J2']

In [23]:
J2_stock_lead = pd.concat([J2_stock_lead1,J2_stock_lead2]).reset_index(drop=True)

In [24]:
J2_stock_lead.shape

(102783, 10)

In [25]:
J3_stock_lead1 = stock_lead1[stock_lead1.Warehouse == 'J3']
J3_stock_lead2 = stock_lead2[stock_lead2.Warehouse == 'J3']

In [26]:
J3_stock_lead = pd.concat([J3_stock_lead1,J3_stock_lead2]).reset_index(drop=True)

In [27]:
J3_stock_lead.shape

(102783, 10)

In [28]:
C2_sales = pd.read_excel('sale_movements/sales_movements_C2.xlsx')

In [29]:
def calculate_buffer(sales, stock_lead):
    sales['EntryDate'] = pd.to_datetime(sales['EntryDate'])
    # only keeping StockCode and TotalLeadTime in lead time data
    stock_lead = stock_lead[['StockCode','TotalLeadTime']].reset_index(drop=True)
    # for each unique stock code, calculate the average TotalLeadTime and maximum TotalLeadTime
    _mean = stock_lead.groupby('StockCode')['TotalLeadTime'].mean()
    _max = stock_lead.groupby('StockCode')['TotalLeadTime'].max()
    stock_lead_meanmax = pd.concat([_mean,_max],axis = 1).reset_index()
    stock_lead_meanmax.columns = ['StockCode','MeanTotalLeadTime','MaxTotalLeadTime']
    sales_lead_time = sales.merge(stock_lead_meanmax,on = 'StockCode',how = 'left')
    TrnQty_sum = sales_lead_time[sales_lead_time.TrnQty > 0].groupby('StockCode')['TrnQty'].sum()
    TrnQty_timerange = sales_lead_time[sales_lead_time.TrnQty > 0].groupby(
    'StockCode')['EntryDate'].apply(lambda x: x.iloc[-1] - x.iloc[0])
    TrnQty_max = sales_lead_time[sales_lead_time.TrnQty > 0].groupby('StockCode')['TrnQty'].max()
    stock_TrnQty_meanmax = pd.concat([TrnQty_sum,TrnQty_timerange,TrnQty_max],axis = 1).reset_index()
    stock_TrnQty_meanmax.columns = ['StockCode','SumTrnQty','timerange','MaxTrnQty']
    stock_TrnQty_meanmax.timerange = stock_TrnQty_meanmax.timerange.dt.days
    stock_TrnQty_meanmax['MeanTrnQty'] = stock_TrnQty_meanmax[
        'SumTrnQty'] / stock_TrnQty_meanmax['timerange']
    stock_TrnQty_meanmax.loc[stock_TrnQty_meanmax[
        'MeanTrnQty'] == np.inf,'MeanTrnQty'] = stock_TrnQty_meanmax.SumTrnQty
    
    TrnQty_leadtime = stock_TrnQty_meanmax.merge(stock_lead_meanmax, on='StockCode',how='left')
    TrnQty_leadtime['BufferStock'] = (TrnQty_leadtime.MaxTrnQty * TrnQty_leadtime.MaxTotalLeadTime) - (
    TrnQty_leadtime.MeanTrnQty * TrnQty_leadtime.MeanTotalLeadTime)
    return TrnQty_leadtime

In [ ]:
# max daily usage - not the maximum turn quantity??? that might be an outlier 

# try validate it with items() first and

# try to eliminate the outlier

# recalculate the MEAN and MAX

In [30]:
def print_metrics(TrnQty_leadtime):
    print("min:", TrnQty_leadtime.BufferStock.min())
    print("25th percentile:", TrnQty_leadtime.BufferStock.quantile(0.25))
    print("median:", TrnQty_leadtime.BufferStock.median())
    print("75th percentile:", TrnQty_leadtime.BufferStock.quantile(0.75))
    print("max:", TrnQty_leadtime.BufferStock.max())
    

In [32]:
df = calculate_buffer(C2_sales, C2_stock_lead)

In [33]:
print_metrics(df)

min: -252.0
25th percentile: 1.5
median: 90.09
75th percentile: 189.43830275229357
max: 16768.59813084112


In [35]:
df.sort_values('BufferStock',ascending = False).head(50)

,StockCode,SumTrnQty,timerange,MaxTrnQty,MeanTrnQty,MeanTotalLeadTime,MaxTotalLeadTime,BufferStock
82,12342PROQC1,4392,107,600,41.046729,30.000000,30.0,16768.598131
119,12972PROQC1,4115,134,500,30.708955,30.000000,30.0,14078.731343
909,3006160,2258,62,500,36.419355,30.000000,30.0,13907.419355
5066,HUFD0210,112,86,74,1.302326,133.500000,186.0,13590.139535
156,212-11B9L-RD-E2,383,138,100,2.775362,125.000000,125.0,12153.079710
910,3009260,910,55,390,16.545455,30.000000,30.0,11203.636364
2389,BD518,962,136,100,7.073529,114.000000,114.0,10593.617647
3768,CVTO1010,1428,140,100,10.200000,93.500000,96.0,8646.300000
5070,HUMZ0040,182,140,40,1.300000,133.500000,186.0,7266.450000
1399,442-1945R-UE,292,139,50,2.100719,125.000000,125.0,5987.410072
